In [1]:
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns

In [13]:
df = pd.read_csv("../data/clean.csv")
# convert date to datetime
df["Date"] = pd.to_datetime(df["Date"])
# select exercises with assistant bands
assistant_band_exercises = df["Weight"].str.isalpha()
assistant_band_exercises = df[assistant_band_exercises]["Exercise"].unique()
df_band = df[df["Exercise"].isin(assistant_band_exercises)]
exercises_band = df_band["Exercise"].unique()
variations_band = df_band["Variation"].unique()

df_non_band = df[~df["Exercise"].isin(exercises_band)]

exercises_non_band = df_non_band["Exercise"].unique()
variations_non_band = df_non_band["Variation"].unique()

In [215]:
print(exercises_band)
print(exercises_non_band)

['FRONT LEVER' 'L SIT' 'PLANCHE' 'BACK LEVER']
['MUSCLE UP' 'PULL UP' 'DEADLIFT' 'FACE PULL' 'BENCH PRESS' 'HIP THRUST'
 'REVERSE BARBELL LUNGE' 'JUMPING ROPE' 'OVERHEAD PRESS' 'SQUAT'
 'CALF RAISE' 'INCLINED BENCH PRESS' 'DIP' 'DEAD HANG' 'HANDSTAND PUSHUP'
 'HYPEREXTENSION' 'TUCK PLANCHE' 'BULGARIAN SPLIT SQUAT' 'DRAGON FLAG'
 'PUSH UP' 'ONE ARM HOLD']


In [216]:
# purple, black, green, orange
band_colour_mapping = {"P": -10, "B": -20, "G": -30, "O": -40}
df_band = df_band.replace({"Weight": band_colour_mapping})

# convert object to float
df_band["Weight"] = pd.to_numeric(df_band["Weight"])
df_non_band["Weight"] = pd.to_numeric(df_non_band["Weight"])

C:\Users\nsc\AppData\Local\Temp\ipykernel_15476\3441532009.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [254]:
def plot_pr(df: pd.DataFrame, exercise: str, variation: str):
    df_ex = df.loc[(df["Exercise"] == exercise) & (df["Variation"] == variation)]
    pr = -np.inf
    date_arr = []
    weight_arr = []
    counts_arr = []

    for date in df_ex["Date"]:
        temp = df_ex.loc[df_ex["Date"] == date]
        max_weight = temp["Weight"].max()

        max_weight_str = temp["Weight"].max()
        counts = temp.loc[temp["Weight"] == max_weight_str]["Count"].max()
        if max_weight > pr:
            pr = max_weight
            date_arr.append(date)
            weight_arr.append(max_weight)
            counts_arr.append(counts)
    if variation == "/":
        variation = ""

    fig = go.Figure(
        [
            go.Scatter(
                x=date_arr,
                y=weight_arr,
                marker=dict(
                    size=counts_arr,
                    sizemode="area",
                    sizeref=2.0 * max(counts_arr) / (40.0**2),
                    sizemin=4,
                ),
            )
        ]
    )
    fig.update_layout(
        xaxis_title="Date", yaxis_title="kg", title=f"{exercise} {variation} PR"
    )
    # <extra></extra> removes the "trace 0" text
    hovertemplate = (
        "Date: %{x}<br>"
        + "Weight: %{y} kg<br>"
        + "Count: %{customdata[0]}"
        + "<extra></extra>"
    )
    fig.update_traces(customdata=pd.DataFrame(counts_arr), hovertemplate=hovertemplate)
    fig.show()

In [252]:
def plot_pr(df: pd.DataFrame, exercise: str, variation: str):
    df_ex = df.loc[(df["Exercise"] == exercise) & (df["Variation"] == variation)]
    pr = -np.inf

    # idx = df_ex.groupby("Date")["Weight"].transform(max) == df_ex["Weight"]
    # df_ex = df_ex[idx]
    # print(df_ex.head())
    weight_cummax = df_ex["Weight"].cummax()
    mask = df_ex["Weight"] >= weight_cummax
    df_result = df_ex[mask]
    print(df_result.head())
    
    if variation == "/":
        variation = ""

    fig = go.Figure(
        [
            go.Scatter(
                x=df_result["Date"],
                y=df_result["Weight"],
                marker=dict(
                    size=df_result["Count"],
                    sizemode="area",
                    sizeref=2.0 * max(df_result["Count"]) / (40.0**2),
                    sizemin=4,
                ),
            )
        ]
    )
    fig.update_layout(
        xaxis_title="Date", yaxis_title="kg", title=f"{exercise} {variation} PR"
    )
    # <extra></extra> removes the "trace 0" text
    hovertemplate = "Date: %{x}<br>" + "Weight: %{y} kg<br>" + "Count: %{customdata[0]}" + "<extra></extra>"
    fig.update_traces(customdata=pd.DataFrame(df_result["Count"]), hovertemplate=hovertemplate)
    fig.show()

In [255]:
plot_pr(df=df_non_band, exercise="BENCH PRESS", variation="/")

In [219]:
def plot_volume(df: pd.DataFrame)

In [15]:
from plots import Plot
from exercise_type import pick_df
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [281]:
df_results = pick_df(df, "BENCH PRESS", "/")

plot = Plot(df_results)
plot.volume_breakdown()

c:\Users\nsc\Desktop\workout-tracker\src\exercise_type.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
plot.volume()

In [306]:
df_body_weight = pd.read_csv("../data/weight.csv", index_col=0)
df_body_weight["Date"] = pd.to_datetime(df_body_weight["Date"], dayfirst=True, errors="coerce")

In [308]:
fig = px.line(df_body_weight, x="Date", y="kg", title="Body Weight")
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_xaxes(range=[df_body_weight["Date"][0],df_body_weight["Date"][5]])

In [310]:
df_non_band["Date"].to_csv("a.csv")

In [316]:
a=pd.read_csv("a.csv")

In [320]:
a["Weight"].str

In [315]:
a

,Exercise,Variation,Weight,Count,Set
Date,,,,,
2023-07-11,MUSCLE UP,/,0,3.0,1
2023-07-11,FRONT LEVER,ONE LEG,-10,5.0,1
2023-07-11,FRONT LEVER,ONE LEG,-10,14.0,2
2023-07-11,FRONT LEVER,ONE LEG,-10,18.0,3
2023-07-11,PULL UP,ARCHER,0,9.0,1
...,...,...,...,...,...
2023-09-12,DIP,/,35,7.0,2
2023-09-12,DIP,/,35,7.0,3
2023-09-14,PULL UP,/,20,6.0,1


In [3]:
import os
a = os.path.dirname(os.path.abspath(__file__))

NameError: name '__file__' is not defined